# Explore Listings Data

The purpose of this notebook is to perform in-depth exploration of the following data sets, transform/clean them if necessary, and make relevant data available data for modeling
* 'data/listings.csv'
    * This file is summary data of listings
* 'data/listings_detailed/listings.csv'
    * This file contains detailed listings data    
* 'data/reviews.csv'
    * This file contains a variable called 'comments', which may represent the complete free-form review
* 'data/prizm_attributes.xlsx'
* Census data extraction using zip codes available in 'data/listings_detailed/listings.csv'

## The output of this notebook is "listings_cleaned.csv"
### This will also include new Census attributes and PRIZM attributes

In [70]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re

## Hypothesis: 
### High capital intensity, high competition, high cost of doing business, and market saturation means growth will come from efficient pricing

## Research Objective:
* How review sentiment varies by price and location
* How neighborhood segments and personas influence rental price
* The drivers of market pricing of properties
* The segmentation of properties

To support the research objectives, this exercise will seek to identify and analyze the following data subject areas:
* Full-text reviews
    * Is sentiment extraction required
* Labeled reviews / Measured reviews (How are provided reviews labeled and measured in the data)
* Analyze the Prizm data in relation to price
    * Clustering of listings based on neighbourhood attributes from Prizm
    * Analysis of variance in price by Clustering
* Assess for other variables in listings that correlate or are not independent from price
* Assess variables and identify those that segment the overall listings data well

## Load Full Listings Data
* 'data/listings_detailed/listings.csv'
* This data set has over 100 columns and 50000 observations

In [71]:
fname = 'data/listings_detailed/listings.csv'
listings = pd.read_csv(fname)

In [72]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)

## When was the Listings Data Prepared?

In [73]:
listings['last_scraped'].value_counts()

2019-12-06    23960
2019-12-05    21722
2019-12-04     4378
2019-12-07      539
Name: last_scraped, dtype: int64

### Listings data was extracted approximately at the beginning of December 2019

### Each observation in this data set represents a unique listing on AirBnb

In [74]:
len(set(listings.id)) == listings.shape[0]

True

## Columns containing price

In [75]:
price_cols = [col for col in listings.columns if re.search('price', col)]

In [76]:
# data type of price
listings[price_cols].dtypes

price            object
weekly_price     object
monthly_price    object
dtype: object

In [77]:
# summary of price
listings[price_cols].describe()

,price,weekly_price,monthly_price
count,50599,5654,4916
unique,647,589,650
top,$100.00,$500.00,"$3,000.00"
freq,2092,279,233


In [78]:
# sample head of price
listings[price_cols].head()

,price,weekly_price,monthly_price
0,$225.00,"$1,995.00",NaN
1,$89.00,$575.00,"$2,100.00"
2,$200.00,NaN,NaN
3,$60.00,NaN,NaN
4,$79.00,$470.00,NaN


### Price columns originally as strings or mixed; these are transformed to numeric

In [79]:
listings['price'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [80]:
listings['weekly_price'].apply(type).unique()

array([<class 'str'>, <class 'float'>], dtype=object)

In [81]:
listings['monthly_price'].apply(type).unique()

array([<class 'float'>, <class 'str'>], dtype=object)

In [82]:
listings['extra_people'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [83]:
# function to clean price values that are string
def clean_string_price(x):
    if isinstance(x, str):
        out = re.sub("[$,]", "", x)
        return out

listings['price'] = listings['price'].apply(clean_string_price).astype('float')
listings['weekly_price'] = listings['weekly_price'].apply(clean_string_price).astype('float')
listings['monthly_price'] = listings['monthly_price'].apply(clean_string_price).astype('float')
listings['extra_people'] = listings['extra_people'].apply(clean_string_price).astype('float')

In [84]:
listings[price_cols].describe()

,price,weekly_price,monthly_price
count,50599.000000,5654.000000,4916.000000
mean,158.171782,890.337283,2932.860252
std,348.089562,799.755626,2539.567001
min,0.000000,85.000000,455.000000
25%,69.000000,470.000000,1500.000000
50%,105.000000,700.000000,2400.000000
75%,175.000000,1050.000000,3500.000000
max,10000.000000,14000.000000,45000.000000


## Assess for Missing Data

In [85]:
check_missing = listings.isna().sum().sort_values(ascending=False)
pd.DataFrame(dict(
    missing_values = check_missing[check_missing > 0],
    proportion_missing = round(check_missing[check_missing > 0] / 50599,2)
))

,missing_values,proportion_missing
thumbnail_url,50599,1.00
medium_url,50599,1.00
xl_picture_url,50599,1.00
host_acceptance_rate,50599,1.00
jurisdiction_names,50583,1.00
license,50577,1.00
square_feet,50213,0.99
monthly_price,45683,0.90
weekly_price,44945,0.89
notes,29976,0.59


### Several variables have a high proportion of missing values (>80%). These will be removed entirely from the analysis.
* Incidentally these include two of the price variables: "monthly_price" and "weekly_price", as well as URLs, host_acceptance_rate, jurisdiction_names, license, and square_feet

## Exclude columns with greater than 80% missing data

In [86]:
vars_excl = ['thumbnail_url','medium_url','xl_picture_url','host_acceptance_rate','jurisdiction_names','license','square_feet','monthly_price','weekly_price']
remove_these = [col for col in vars_excl if col in listings.columns]
if set(remove_these).issubset(listings.columns):
    listings = listings.drop(remove_these, axis=1)

## Assess for uniqueness in columns

In [87]:
constant_cols = list(set(listings.columns) - set(listings.loc[:, (listings != listings.iloc[0]).any()].columns))

In [88]:
listings[constant_cols]

,scrape_id,country_code,requires_license,experiences_offered,country,has_availability,is_business_travel_ready
0,20191204162729,US,f,none,United States,t,f
1,20191204162729,US,f,none,United States,t,f
2,20191204162729,US,f,none,United States,t,f
3,20191204162729,US,f,none,United States,t,f
4,20191204162729,US,f,none,United States,t,f
...,...,...,...,...,...,...,...
50594,20191204162729,US,f,none,United States,t,f
50595,20191204162729,US,f,none,United States,t,f
50596,20191204162729,US,f,none,United States,t,f
50597,20191204162729,US,f,none,United States,t,f


### The columns 'has_availability', 'experiences_offered', 'country', 'country_code', 'is_business_travel_ready', 'scrape_id', and 'requires_license' only contain *unique values*.
* These columns will be removed for the purposes of this activity

In [89]:
remove_these = [col for col in constant_cols if col in listings.columns]
if set(remove_these).issubset(listings.columns):
    listings = listings.drop(remove_these, axis=1)

## Inspect the data types for the rest of the columns and exclude those that don't make sense for regression or segmentation analysis

In [90]:
# id might be useful, but not as a variable. Set as index
listings = listings.set_index('id')

In [91]:
# any columns that are URLs can be removed
url_cols = [col for col in listings.columns if re.search('url', col)]
remove_these = [col for col in url_cols if col in listings.columns]
if set(remove_these).issubset(listings.columns):
    listings = listings.drop(remove_these, axis=1)

In [92]:
# any columns with mention of the word 'scrape' can be removed as it is a reference to the data collection
scrape_cols = [col for col in listings.columns if re.search('scrape', col)]
remove_these = [col for col in scrape_cols if col in listings.columns]
if set(remove_these).issubset(listings.columns):
    listings = listings.drop(remove_these, axis=1)

In [93]:
# look for free-text columns that are either redundant or long strings of text
dfstr = listings.loc[:, listings.dtypes == 'object']

In [94]:
# function to apply over column and get average length of strings
def getavg(col):
    return col.astype(str).map(len).mean()
# get average length of strings in each column
# dfstr.apply(getavg, axis=0).sort_values(ascending = False)

In [95]:
# for the purposes of this analysis, columns for long text descriptions will be removed
# some others such as host_location and street will also be removed as neighborhood details for segmentation will not be excluded
# likewise, latitude and longitude aren't necessary to keep as zip codes are available
# remove id columns and host name
# remove security deposit and cleaning fee (>25% missing values)
desc_cols = ['description','space','summary','amenities','neighborhood_overview','house_rules','host_about','transit','notes','interaction','access','host_verifications','name','host_location','street','latitude','longitude','id','host_id', 'host_name', 'security_deposit','cleaning_fee']
remove_these = [col for col in desc_cols if col in listings.columns]
if set(remove_these).issubset(listings.columns):
    listings = listings.drop(remove_these, axis=1)

In [96]:
dfstr = listings.loc[:, listings.dtypes == 'object']
# dfstr.apply(lambda x: x[:3], axis=0).transpose()

## Transform Dates
* Variables 'host_since', 'first_review', and 'last_review' need to be converted from string to date 
* Additional variables created that might be useful for regression and segmentation analysis:
    * 'host_tenure': difference in days between 'host_since' and max('host_since') (based on available data)
    * 'review_duration': difference in days between 'last_review' and 'first_review' (based on available data)

In [97]:
listings['host_since'] = pd.to_datetime(listings['host_since'], errors = 'coerce')
listings['first_review'] = pd.to_datetime(listings['first_review'], errors = 'coerce')
listings['last_review'] = pd.to_datetime(listings['last_review'], errors = 'coerce')
listings[['host_since','first_review','last_review']].head()

,host_since,first_review,last_review
id,,,
2595,2008-09-09,2009-11-21,2019-11-04
3831,2008-12-07,2014-09-30,2019-11-22
5099,2009-02-02,2009-04-20,2019-10-13
5121,2009-02-03,2009-05-28,2017-10-05
5178,2009-03-03,2009-05-06,2019-11-21


In [98]:
listings['host_tenure'] = listings['host_since'].max() - listings['host_since']
listings['host_tenure'] = [d.days for d in listings['host_tenure']]
listings['review_duration'] = listings['last_review'] - listings['first_review']
listings['review_duration'] = [d.days for d in listings['review_duration']]
listings[['host_tenure','review_duration']].head()

,host_tenure,review_duration
id,,
2595,4102.0,3635.0
3831,4013.0,1879.0
5099,3956.0,3828.0
5121,3955.0,3052.0
5178,3927.0,3851.0


## Clean Zipcode Variable
* It is possible that several variables require more thorough cleaning prior to modeling, but this notebook will focus primarily on cleaning the zipcode variable
* Regular expression was used to extract a 5-digit (New York City) zip code.

In [99]:
print(set(listings['zipcode']))

{'11238', nan, '10013', '11234', '11372', '11354', '10457', '11208', '11358', 'NY 10007', 'NY 10023', '10270', '10474', '10024', '10038', '10044', '11249\n11249', '11412', '11429', '10470', '11433', 'NY 10014', 'NY 10028', '11691', '10460', '11249', '11203', '11230', '10012', 'NY 10006', '11216', '11233', '10458', '11428', '10035', '10032', '11374', '11355', '10463', '11217', '11694', '11385-2308', 'NY 10002', '10471', '11232', '10019', '10314', '11369', '11360', '10469', '11225', '11434', '11213', '10282', '11365', '11103', '11693', 'NY 10022', '10304', '10705', 'NY 10018', '11421', '11420', '10306', 'NY 10012', 'NY 10024', '11226', '11435', '10028', '10005', '11368', '11235', 'NY 10011', '10464', '11427', '11362', '11215', '10001', '10472', '11436', 'NY 11249', '11361', '11109', '10305', '10467', '11239', '10453', '10129', '11377', '10312', '10281', '10022', '10002-2289', '11426', '10007', '10031', '10010', '10069', '10459', '11366', '11367', '10280', '11231', '11224', '10162', '1046

In [100]:
listings['zipcode'] = listings['zipcode'].str.extract(r'(\d{5})')

## Assess correlation between numeric variables

In [101]:
cormat = listings.corr()
cormatabs = cormat.abs()
s = cormatabs.unstack()
so = s.sort_values(kind="quicksort", ascending = False).loc[s < 1]

### High pair-wise correlations

In [102]:
so.loc[so >= 0.7]

maximum_nights_avg_ntm                       minimum_maximum_nights                         0.999869
minimum_maximum_nights                       maximum_nights_avg_ntm                         0.999869
calculated_host_listings_count_entire_homes  calculated_host_listings_count                 0.978691
calculated_host_listings_count               calculated_host_listings_count_entire_homes    0.978691
availability_90                              availability_60                                0.975593
availability_60                              availability_90                                0.975593
maximum_minimum_nights                       minimum_nights_avg_ntm                         0.961899
minimum_nights_avg_ntm                       maximum_minimum_nights                         0.961899
minimum_minimum_nights                       minimum_nights                                 0.960256
minimum_nights                               minimum_minimum_nights                        

### There are many variables in the dataframe that are highly correlated with one another. These should be addressed during regression modeling to mitigate the effects of multicollinearity.

## Correlations between numeric variables and price

In [103]:
cormat['price'].sort_values().head()

number_of_reviews_ltm                          -0.044478
number_of_reviews                              -0.041875
calculated_host_listings_count_shared_rooms    -0.037092
calculated_host_listings_count_private_rooms   -0.030891
host_tenure                                    -0.025922
Name: price, dtype: float64

### It doesn't appear that any of the numeric variables are *highly correlated* with price (i.e. > 0.7), but there appear to be some interesting results
* Number of reviews is negatively correlated with price and appears to indicate that there are generally more reviews for cheaper listings
* Variables that show positive correlation with price include:
    * **number of guests included, bathrooms, beds, bedrooms, and the number of persons that can be accommodated**

## Extract and cluster PRIZM features
* Zip Codes are used to obtain PRIZM features from https://claritas360.claritas.com/mybestsegments/?_ga=2.191688681.2124823222.1579060196-470179875.1579060196#zipLookup
* DBSCAN (density-based clustering) is used to get clusterings of listings based on their PRIZM features

In [104]:
# get PRIZM features by zipcode from excel file
fname = 'data/prizm_attributes.xlsx'
prizm = pd.read_excel(fname, sheet_name='zipcodes', header=None)
prizm = prizm.dropna() # remove rows without features
prizm.columns = ['zipcode','features']

In [105]:
# dictionary of features by zipcode
d = prizm.set_index('zipcode').T.to_dict('list')

In [106]:
# function to process features into a list of strings
def tolist(v):
    v = v[0]
    return [x for x in v.split(',')]

In [107]:
# clean dictionary values
for k in d.keys():
    d[k] = tolist(d[k])

In [108]:
# print some items
some_keys = list(d.keys())[:5]
for k in some_keys: 
    print(k,d[k])  

11211 ['31', '17', '40', '56', '4']
11221 ['42', '40', '63', '31', '17']
11206 ['63', '31', '42', '40', '56']
11216 ['40', '17', '31', '21', '63']
10019 ['17', '31', '4', '40', '7']


In [109]:
# prizm feature definitions
prizm_definitions = pd.read_excel(fname, sheet_name='attributes')
print("Sample:\n")
prizm_definitions.head()

Sample:



,id,rollup,description
0,1,Upper Crust,Wealthy Mature w/o Kids
1,2,Networked Neighbors,Wealthy Middle Age Mostly w/ Kids
2,3,Movers & Shakers,Wealthy Older Mostly w/o Kids
3,4,Young Digerati,Wealthy Younger Mostly w/ Kids
4,6,Winner's Circle,Wealthy Middle Age Mostly w/ Kids


## Create Prizm Featureset

In [110]:
print("These are the PRIZM features for New York zip codes:\n")
prizm_features = prizm_definitions['id'].astype('str').tolist()
print(prizm_features)

These are the PRIZM features for New York zip codes:

['1', '2', '3', '4', '6', '7', '8', '12', '13', '17', '19', '20', '21', '22', '24', '25', '31', '33', '35', '36', '40', '42', '43', '45', '49', '56', '59', '63', '64']


In [111]:
# these are the documents for which we need featureset
zip_df = pd.DataFrame(listings['zipcode']).dropna()
zip_df['zipcode'] = pd.to_numeric(zip_df['zipcode'])
zip_df = zip_df[zip_df['zipcode'].isin(d.keys())]
zip_df['features'] = [d[k] for k in zip_df['zipcode'].values]

In [112]:
# function to create a prizm featureset
# param: document a list of features for a zipcode
def document_features(document, prizm_features):
    document_words = set(document)
    features = {}
    for f in prizm_features:
        features['V_{}'.format(f)] = (f in document_words)
    return features
# get feature sets for all zip codes in zip_df
# documents is a list of lists where each list is a feature in zip_df
documents = zip_df['features'].tolist()
featuresets = [document_features(d, prizm_features) for d in documents]

In [113]:
# data frame for clustering
df_clust = pd.DataFrame(featuresets).set_index(zip_df['zipcode'])
df_clust

,V_1,V_2,V_3,V_4,V_6,V_7,V_8,V_12,V_13,V_17,...,V_36,V_40,V_42,V_43,V_45,V_49,V_56,V_59,V_63,V_64
zipcode,,,,,,,,,,,,,,,,,,,,,
10018,False,False,False,True,False,True,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
11238,False,False,False,True,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
10016,False,False,False,True,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
11216,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
10019,False,False,False,True,False,True,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10458,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,True,False,True,False
11105,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
10458,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,True,False,True,False


## Density-based clustering of PRIZM features with DBSCAN
* This is a density-based clustering technique that automatically detects a clustering. The following parameters were used:
    * eps (epsilon radius) = 0.5
    * min_samples = 5

In [114]:
from sklearn.cluster import DBSCAN
# run the DBSCAN algorithm on data to construct a DBSCAN object
db = DBSCAN(eps=0.5, min_samples=5).fit(df_clust)

In [115]:
# cluster labels
labels = db.labels_
# number of clusters in labels, ignoring noise if present
n_clusters_ = len(set(labels))
print('Estimated number of clusters: {:d}'.format(n_clusters_))

Estimated number of clusters: 81


In [116]:
unique, counts = np.unique(labels, return_counts = True)
clustering = dict(zip(unique, counts))
clustering = [x for x in clustering.values()]

In [117]:
print("Top 10 Clusters:")
pd.DataFrame(dict(label = unique, size = counts)).sort_values(by = 'size', ascending = False).head(10)

Top 10 Clusters:


,label,size
5,4,5981
1,0,5622
2,1,5202
10,9,3807
7,6,3305
14,13,2677
3,2,2490
6,5,2206
23,22,1621
15,14,1538


In [118]:
print("DBSCAN was not able to assign {:d} listings to a cluster. These listings are considered outliers.".format(clustering[0]))

DBSCAN was not able to assign 15 listings to a cluster. These listings are considered outliers.


## Overlay Clusters to listings

In [119]:
zip_df['prizm_cluster'] = labels
zip_df2 = zip_df.drop(['zipcode','features'], axis = 1)

In [120]:
# merge listings with clusters
listings2 = listings.join(zip_df2)

In [121]:
# function to convert cluster to string label
re.findall(pattern = "(\d+).", string = "22.0")[0]
def getstr(x):
    try:
        out = re.findall(pattern = "(\d+).", string = str(x))[0]
    except:
        out = 'UNK'
    return out

In [122]:
listings2['prizm_cluster'] = listings2['prizm_cluster'].apply(lambda x: getstr(x))

### Dictionary of prizm features converted to words

In [123]:
prizm_definitions.head()

,id,rollup,description
0,1,Upper Crust,Wealthy Mature w/o Kids
1,2,Networked Neighbors,Wealthy Middle Age Mostly w/ Kids
2,3,Movers & Shakers,Wealthy Older Mostly w/o Kids
3,4,Young Digerati,Wealthy Younger Mostly w/ Kids
4,6,Winner's Circle,Wealthy Middle Age Mostly w/ Kids


In [124]:
# dictionary of features by zipcode
d2 = prizm_definitions[['id','rollup']].set_index('id').T.to_dict('list')

In [125]:
featurelist = zip_df['features'].tolist()
featurelistnew = []
for fvlist in featurelist:
    fvlistnew = []
    for i in fvlist:
        fvlistnew.extend(d2[int(i)])
    featurelistnew.append(str(fvlistnew))

In [126]:
zip_df['prizm_features'] = featurelistnew

In [127]:
zip_df.head()

,zipcode,features,prizm_cluster,prizm_features
id,,,,
2595,10018,"[31, 17, 4, 40, 7]",0,"['Connected Bohemians', 'Urban Elders', 'Young..."
3831,11238,"[31, 17, 40, 4, 21]",1,"['Connected Bohemians', 'Urban Elders', 'Aspir..."
5099,10016,"[31, 17, 4, 40, 21]",1,"['Connected Bohemians', 'Urban Elders', 'Young..."
5121,11216,"[40, 17, 31, 21, 63]",2,"['Aspiring A-Listers', 'Urban Elders', 'Connec..."
5178,10019,"[17, 31, 4, 40, 7]",0,"['Urban Elders', 'Connected Bohemians', 'Young..."


In [128]:
# merge listings with prizm features
listings3 = listings2.join(zip_df['prizm_features'])
print("New Prizm Attributes added to listings:\n")
listings3[['prizm_cluster','prizm_features']]

New Prizm Attributes added to listings:



,prizm_cluster,prizm_features
id,,
2595,0,"['Connected Bohemians', 'Urban Elders', 'Young..."
3831,1,"['Connected Bohemians', 'Urban Elders', 'Aspir..."
5099,1,"['Connected Bohemians', 'Urban Elders', 'Young..."
5121,2,"['Aspiring A-Listers', 'Urban Elders', 'Connec..."
5178,0,"['Urban Elders', 'Connected Bohemians', 'Young..."
...,...,...
40570168,22,"['Low-Rise Living', 'Multi-Culti Mosaic', 'Mul..."
40570350,2,"['Urban Elders', 'Connected Bohemians', 'The C..."
40570935,22,"['Low-Rise Living', 'Multi-Culti Mosaic', 'Mul..."


## Extract Census Data Using Listing Zip Codes
* https://pypi.org/project/uszipcode/
* 'population','population_density','housing_units','occupied_housing_units','median_home_value','median_household_income'

In [129]:
# Get these Census attributes
attr = ['population','population_density','housing_units','occupied_housing_units','median_home_value','median_household_income']

In [130]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

In [131]:
# function to get census data for zipcode
# param: zip string or integer zipcode
# param: what string of key value e.g. 'population','population_density','housing_units','occupied_housing_units','median_home_value','median_household_income'
def getcensus(zip, what):
    try:
        out = search.by_zipcode(zip).to_dict()[what]
        return out
    except:
        pass

In [132]:
zip_df3 = pd.DataFrame({
    'zipcode' : zip_df['zipcode'].drop_duplicates().values
})

In [133]:
for a in attr:
    zip_df3[a] = zip_df3['zipcode'].apply(lambda x: getcensus(x, a))

In [134]:
zip_df3.head()

,zipcode,population,population_density,housing_units,occupied_housing_units,median_home_value,median_household_income
0,10018,5229.0,16251.0,4425.0,3065.0,1000001.0,104635.0
1,11238,49262.0,44555.0,24525.0,22648.0,656800.0,65315.0
2,10016,54183.0,101814.0,35617.0,32173.0,759600.0,105324.0
3,11216,54316.0,58065.0,25964.0,23193.0,623300.0,43996.0
4,10019,42870.0,63042.0,31404.0,25905.0,808600.0,84424.0


In [135]:
zip_df3['zipcode'] = zip_df3['zipcode'].astype('object')
census_dict = zip_df3.set_index('zipcode').T.to_dict('dict')

In [136]:
# function to get census data for zipcode
# param: zip string or integer zipcode
# param: what string of key value e.g. 'population','population_density','housing_units','occupied_housing_units','median_home_value','median_household_income'
def getcensus2(z, what):
    try:
        z = int(z)
        out = census_dict[z][what]
        return out
    except:
        pass

In [137]:
listings4 = listings3
for a in attr:
    listings4[a] = [getcensus2(z, a) for z in listings4['zipcode'].values]
print("New Census Attributes added to listings:\n")
listings4[attr] 

New Census Attributes added to listings:



,population,population_density,housing_units,occupied_housing_units,median_home_value,median_household_income
id,,,,,,
2595,5229.0,16251.0,4425.0,3065.0,1000001.0,104635.0
3831,49262.0,44555.0,24525.0,22648.0,656800.0,65315.0
5099,54183.0,101814.0,35617.0,32173.0,759600.0,105324.0
5121,54316.0,58065.0,25964.0,23193.0,623300.0,43996.0
5178,42870.0,63042.0,31404.0,25905.0,808600.0,84424.0
...,...,...,...,...,...,...
40570168,79492.0,78784.0,27806.0,26201.0,330300.0,24618.0
40570350,36688.0,22481.0,17416.0,16120.0,658700.0,57525.0
40570935,79492.0,78784.0,27806.0,26201.0,330300.0,24618.0


In [138]:
# listings cleaned includes new Census attributes and PRIZM attributes
# this dataset has also undergone some cleaning and feature selection.
listings4.to_csv('data/listings_cleaned.csv')

### Listings cleaned includes new Census attributes and PRIZM attributes this dataset has also undergone some cleaning and feature selection.